## Demo Notebook of Function Calling with liteLLM
- Supported Providers for Function Calling
  - OpenAI - `gpt-4-0613` and `gpt-3.5-turbo-0613`
- In this notebook we use function calling with `litellm.completion()`

In [ ]:
## Install liteLLM
!pip install litellm

In [1]:
import os, litellm
from litellm import completion

/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.6_1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
os.environ['OPENAI_API_KEY'] = "" #@param

## Define Messages, Functions
We create a get_current_weather() function and pass that to GPT 3.5

See OpenAI docs for this: https://openai.com/blog/function-calling-and-other-api-updates

In [2]:
messages = [
    {"role": "user", "content": "What is the weather like in Boston?"}
]

def get_current_weather(location):
  if location == "Boston, MA":
    return "The weather is 12F"

functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]

## Call gpt-3.5-turbo-0613 to Decide what Function to call

In [3]:
response = completion(model="gpt-3.5-turbo-0613", messages=messages, functions=functions)
print(response)


ModelResponse(id='chatcmpl-8aUK2PmRBKdydoUys8KwYRRgFP250', choices=[Choices(finish_reason='function_call', index=0, message=Message(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')))], created=1703707057, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=Usage(completion_tokens=18, prompt_tokens=82, total_tokens=100))


In [ ]:

#experimental function calling using guidance
import os, litellm
from litellm import completion

messages = [
    {"role": "user", "content": "What is the weather like in Boston?"}
]

def get_current_weather(location):
  if location == "Boston, MA":
    return "The weather is 12F"

functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
litellm.add_function_to_prompt = True
litellm.set_verbose=True
response = completion(model="bedrock/anthropic.claude-instant-v1", messages=messages, functions=functions)
print(response["choices"][0]["message"]["function_call"])

## Parse GPT 3.5 Response
Read Information about what Function to Call

In [3]:
function_call_data = response["choices"][0]["message"]["function_call"]
function_call_data

FunctionCall(arguments='{"location": "Boston, MA"}', name='get_current_weather')

In [6]:
import json
function_name = function_call_data.name
function_args = function_call_data.arguments
function_args = json.loads(function_args)
print(function_name, function_args)


get_current_weather {'location': 'Boston, MA'}


## Call the get_current_weather() function

In [7]:
if function_name == "get_current_weather":
  result = get_current_weather(**function_args)
  print(result)

The weather is 12F


## Send the response from get_current_weather back to the model to summarize

In [26]:
messages = [
    {"role": "user", "content": "What is the weather like in Boston?"},
    {"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": "{ \"location\": \"Boston, MA\"}"}},
    {"role": "function", "name": "get_current_weather", "content": result}
]
response = completion(model="gpt-3.5-turbo-0613", messages=messages, functions=functions)
print(response)

{
  "id": "chatcmpl-7mXGN62u75WXp1Lgen4iSgNvA7hHT",
  "object": "chat.completion",
  "created": 1691801963,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston is 12 degrees Fahrenheit."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 109,
    "completion_tokens": 12,
    "total_tokens": 121
  }
}
